<a href="https://www.kaggle.com/code/sharanharsoor/pyspark-build-a-simple-ml-model?scriptVersionId=122397132" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


In [2]:
# AS this notebook for developing a ML pipeline using spark. installing pyspark.
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824024 sha256=99199f57521d9c3edad59f05d6ec54e876a3e612d349452e89f8a4b5724a5be9
  Stored in directory: /root/.cache/pip/wheels/07/fb/67/b9f2c0242d156eaa136b45ae4fd99d3e7c0ecc2acfd26f47b9
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


# Loading libraries

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.appName('logistic_reg').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/16 20:37:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# DataFrame
Reading Dataframe :

Spark ML, allows a variety of data sources to be used such as relational tables, CSVs, etc. For example, a DataFrame could have different columns storing text, feature vectors, true labels, and predictions.



In [4]:
from sklearn import datasets
df = spark.read.csv('../input/iris/Iris.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



In [5]:
df.show(3)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 3 rows



# Feature Transformers

## StringIndexer
StringIndexer encodes a string column of labels to a column of label indices. StringIndexer can encode multiple columns.

In [6]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='Species', outputCol='speciesIndex')
iris = indexer.fit(df).transform(df)
iris.sample(fraction=0.1).show(3)

+---+-------------+------------+-------------+------------+-----------+------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|speciesIndex|
+---+-------------+------------+-------------+------------+-----------+------------+
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|         0.0|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|         0.0|
| 19|          5.7|         3.8|          1.7|         0.3|Iris-setosa|         0.0|
+---+-------------+------------+-------------+------------+-----------+------------+
only showing top 3 rows



## OneHotEncoder
One-hot encoding maps a categorical feature, represented as a label index, to a binary vector with at most a single one-value indicating the presence of a specific feature value from among the set of all feature values. This encoding allows algorithms that expect continuous features, such as Logistic Regression, to use categorical features. For string-type input data, it is common to encode categorical features using StringIndexer first.

In [7]:
from pyspark.ml.feature import OneHotEncoder

encoded = OneHotEncoder(inputCol="speciesIndex", outputCol="species_vec")
new_df = encoded.fit(iris).transform(iris)
new_df.show(3)

+---+-------------+------------+-------------+------------+-----------+------------+-------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|speciesIndex|  species_vec|
+---+-------------+------------+-------------+------------+-----------+------------+-------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|         0.0|(2,[0],[1.0])|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|         0.0|(2,[0],[1.0])|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|         0.0|(2,[0],[1.0])|
+---+-------------+------------+-------------+------------+-----------+------------+-------------+
only showing top 3 rows



## Vector Assembler
VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees. VectorAssembler accepts the following input column types: all numeric types, a boolean type, and a vector type. In each row, the values of the input columns will be concatenated into a vector in the specified order.




In [8]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], 
    outputCol="features")

dataset = assembler.transform(new_df)
dataset.show(3)

+---+-------------+------------+-------------+------------+-----------+------------+-------------+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|speciesIndex|  species_vec|         features|
+---+-------------+------------+-------------+------------+-----------+------------+-------------+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|         0.0|(2,[0],[1.0])|[5.1,3.5,1.4,0.2]|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|         0.0|(2,[0],[1.0])|[4.9,3.0,1.4,0.2]|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|         0.0|(2,[0],[1.0])|[4.7,3.2,1.3,0.2]|
+---+-------------+------------+-------------+------------+-----------+------------+-------------+-----------------+
only showing top 3 rows



In [9]:
dataset.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)
 |-- speciesIndex: double (nullable = false)
 |-- species_vec: vector (nullable = true)
 |-- features: vector (nullable = true)



# Estimator
An estimator is a stage of the learning algorithm that fits a model on a dataset. The whole process can be denoted as follows DataFrame =[fit]=> Model. An estimator is executed during the step of building a model using existing data. A model, which will be made by the estimator, itself serves as a transformer. Further, an estimator is available in Java through the Estimator class. The .fit() method, which is used for building a model, is available through this class.

In [10]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features', labelCol='speciesIndex')
trainData, testData = dataset.randomSplit([0.7, 0.3])

In [11]:
model = lr.fit(trainData)
summary = model.evaluate(testData)

# Evaluator
evaluation stage of the built model.

In [12]:
summary.accuracy

1.0

# Pipeline in Spark:
A pipeline is a series of activities or transformations. Machine learning is performed by implementing a set of tasks, that is collecting the data, cleaning the data, building the model, evaluating the model, etc. All these steps need to be performed one after the other in a particular sequence. In short, a pipeline can be considered as a series of activities wrapped together for better representation. Pipelines primarily have four principal objects- DataFrame, Transformer, Estimator, and Evaluator.

In [13]:
#Using Pipeline #import module
from pyspark.ml import Pipeline

df = spark.read.csv('../input/iris/Iris.csv', header=True, inferSchema=True)

#feature transformers
indexer = StringIndexer(inputCol='Species', outputCol='speciesIndex')

#Apply OneHotEncoder to Species column
encoded = OneHotEncoder(inputCol="speciesIndex", outputCol="species_vec")

#Merge multiple columns into a vector column
assembler = VectorAssembler(
    inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], 
    outputCol="features")

#model
lr = LogisticRegression(featuresCol='features', labelCol='speciesIndex')

#Create pipeline and pass it to stages
pipeline = Pipeline(stages=[ indexer, encoded,
           assembler, lr
])

trainData, testData = df.randomSplit([0.7, 0.3])

#Use .fit() and .transform() on the pipeline
df_transformed = pipeline.fit(trainData).transform(trainData)

df_transformed.select("species", "features", "probability", "prediction").show(3)


+-----------+-----------------+--------------------+----------+
|    species|         features|         probability|prediction|
+-----------+-----------------+--------------------+----------+
|Iris-setosa|[5.1,3.5,1.4,0.2]|[2.92739875492907...|       1.0|
|Iris-setosa|[4.9,3.0,1.4,0.2]|[2.64881336668164...|       1.0|
|Iris-setosa|[4.6,3.1,1.5,0.2]|[6.09823587030075...|       1.0|
+-----------+-----------------+--------------------+----------+
only showing top 3 rows



In [14]:
# End of the notebook.